# Data Processing Assignment

#### Importing Custom Libraries 

I created custom functions to deal with the most common data issues.

1. The first function is the completeness ratio function, which assesses the completeness of the dataset by calculating the proportion of complete to missing data for each variable.

2. The second function, data_val_erroneous checks if all the data types within a column are consistent. For example, it flags if a column with numerical data has observations with text data instead.

3. The third function, data_val_duplicates simply returns the number of duplicate rows in the dataset where the entire row is identical.

In [1]:
import pandas as pd
from Data_Validation import completness_ratio, data_val_erroneous, data_val_duplicates
from functools import reduce

#### Importing Datasets

In [2]:
df_feed = pd.read_csv("feed-data.csv")
df_output = pd.read_csv("output-data.csv")

#### Data Processing Pipeline
- Merging 
- Identify missing or incorrect data
- Return data file that can be used for further analysis

#### Data Assumptions I Made

- Run ID serves as the primary key, enabling the linkage between the two datasets by connecting the experimental input data with the output results.

 - The Date in both datasets refers to the date the experiment was conducted, ensuring alignment of data across both sources.

- Missing weight data (either before or after drying) represents a critical issue that could significantly impact the validity of post-experiment analysis. On the other hand, missing additive measurements, while reducing the granularity of information, still allow for valuable insights to be derived from the remaining data.


In [3]:
merged_data = pd.merge(df_feed, df_output, left_on='Run ID', right_on='Run', how='inner')


In [4]:
def handle_missing_values(df):
    "Removes rows with missing values expect for the Additive columns"
    additive_columns = ['Additive 1 (g)', 'Additive 2 (g)', 'Additive 3 (g)']
    clean_df = df.dropna(subset=[col for col in df.columns if col not in additive_columns])
    return clean_df

In [5]:
def combine_date_columns(df):
    """If the dates in both columns differ, the date from the input file (Date_x) is used."""
    df['Date'] = df.apply(lambda row: row['Date_x'] if row['Date_x'] != row['Date_y'] else row['Date_y'], axis=1)
    df = df.drop(columns=['Date_x', 'Date_y'])
    return df

In [6]:
pipeline = [handle_missing_values, combine_date_columns]
df = reduce(lambda acc, f: f(acc), pipeline, merged_data)
df.to_csv("Cleaned_Data.csv", index=False)

C:\Users\haith\AppData\Local\Temp\ipykernel_18656\3739832262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = df.apply(lambda row: row['Date_x'] if row['Date_x'] != row['Date_y'] else row['Date_y'], axis=1)


#### Generate Report Highlighting Data Quality Assessment and Issues

In [7]:
def data_quality_report(df, output_file='data_quality_report.txt'):
    """This function creates a textual data quality report and saves it to a text file."""
    #Data Completness Summary
    completeness_df = completness_ratio(df)
    completeness_report = "Data Completeness:\n"
    completeness_report += "------------------\n"
    for idx, row in completeness_df.iterrows():
        completeness_report += f"- {row['Variable']}: {row['Completeness_Ratio']:.2f}% complete\n"
    
    # Data Type Consistency Summary
    data_type_consistency_report = data_val_erroneous(df)
    data_type_report = "Data Type Consistency:\n"
    data_type_report += "----------------------\n"
    data_type_report += data_type_consistency_report + "\n"
    
    # Date Mismatch Summary
    df['Date_feed'] = pd.to_datetime(df['Date_x'], format='%d/%m/%Y')
    df['Date_output'] = pd.to_datetime(df['Date_y'], format='%d/%m/%Y')
    date_mismatch = (df['Date_feed'] != df['Date_output']).sum()
    date_mismatch_report = "Date Mismatch:\n"
    date_mismatch_report += "--------------\n"
    date_mismatch_report += f"There are {date_mismatch} rows with mismatched dates between the feed data and output data files.\n"
    
    # Duplicates Summary
    duplicates_report = data_val_duplicates(df)
    duplicates_section = "Duplicates:\n"
    duplicates_section += "-----------\n"
    duplicates_section += duplicates_report + "\n"
    
    # Building the report
    report = "Data Quality Report\n"
    report += "===================\n\n"
    report += completeness_report + "\n"
    report += data_type_report + "\n"
    report += date_mismatch_report + "\n"
    report += duplicates_section
    
    with open(output_file, 'w') as file:
        file.write(report)
    
    return report

In [8]:
print(data_quality_report(merged_data))

Data Quality Report

Data Completeness:
------------------
- Run ID: 100.00% complete
- Date_x: 99.66% complete
- Process Start Time: 94.22% complete
- Feed Type: 100.00% complete
- Feed (g): 93.20% complete
- Additive 1 (g): 91.84% complete
- Additive 2 (g): 91.84% complete
- Additive 3 (g): 20.41% complete
- Run: 100.00% complete
- Date_y: 99.32% complete
- Weight before Drying: 92.18% complete
- Weight after Drying: 71.09% complete
- Output A (g): 72.11% complete
- Speed (RPM): 96.94% complete

Data Type Consistency:
----------------------
Inconsistent data types found: Column 'Feed (g)' may have inconsistent data types: Numerical and Unknown

Date Mismatch:
--------------
There are 4 rows with mismatched dates between the feed data and output data files.

Duplicates:
-----------
 The dataset contains 0 duplicates

